## Python通过折线图发现产品流量问题

折线图：显示随时间而变化的连续数据，展示在相等时间间隔下数据的趋势。

实例：数据来自kaggle网站的"E-commerce website Funnel analysis"  
地址为：https://www.kaggle.com/aerodinamicc/ecommerce-website-funnel-analysis

网站很简单，有四个页面数据：
1. home_page_table.csv，首页用户访问数据
2. search_page_table.csv，搜索页用户访问数据
3. payment_page_table.csv，支付信息页用户访问数据
4. payment_confirmation_table.csv，支付成功页用户访问数据
5. user_table.csv，用户信息数据

目标：绘制转化率的折线图，查看是否有异常情况

### 1. 读取数据

In [1]:
import pandas as pd
import numpy as np

import pyecharts.options as opts
from pyecharts.charts import Line

#### 读取5个数据表到df

In [2]:
df_home_page = pd.read_csv("./datas/ecommerce-website-funnel-analysis/home_page_table.csv")
df_search_page = pd.read_csv("./datas/ecommerce-website-funnel-analysis/search_page_table.csv")
df_payment_page = pd.read_csv("./datas/ecommerce-website-funnel-analysis/payment_page_table.csv")
df_payment_confirmation_page = pd.read_csv("./datas/ecommerce-website-funnel-analysis/payment_confirmation_table.csv")
df_user_table = pd.read_csv("./datas/ecommerce-website-funnel-analysis/user_table.csv")

#### 关联5个数据表为一个大表

In [3]:
df_merge = df_user_table

for df_inter in [df_home_page, df_search_page, df_payment_page, df_payment_confirmation_page]:
    # 每次循环都会忘df_merge中添加新列
    df_merge = pd.merge(
        left=df_merge, 
        right=df_inter, 
        left_on="user_id", 
        right_on="user_id", 
        how="left"
    )

In [4]:
df_merge.head(3)

,user_id,date,device,sex,page_x,page_y,page_x,page_y
0,450007,2015-02-28,Desktop,Female,home_page,NaN,NaN,NaN
1,756838,2015-01-13,Desktop,Male,home_page,NaN,NaN,NaN
2,568983,2015-04-09,Desktop,Male,home_page,search_page,NaN,NaN


In [5]:
df_merge.columns = [
    "user_id", "date", "device", "sex", 
    "home_page", "search_page", "payment_page", "confirmation_page"]
df_merge.head(3)

,user_id,date,device,sex,home_page,search_page,payment_page,confirmation_page
0,450007,2015-02-28,Desktop,Female,home_page,NaN,NaN,NaN
1,756838,2015-01-13,Desktop,Male,home_page,NaN,NaN,NaN
2,568983,2015-04-09,Desktop,Male,home_page,search_page,NaN,NaN


In [6]:
df_merge["date"] = pd.to_datetime(df_merge["date"])
df_merge.head(3)

,user_id,date,device,sex,home_page,search_page,payment_page,confirmation_page
0,450007,2015-02-28,Desktop,Female,home_page,NaN,NaN,NaN
1,756838,2015-01-13,Desktop,Male,home_page,NaN,NaN,NaN
2,568983,2015-04-09,Desktop,Male,home_page,search_page,NaN,NaN


### 2. 展现每个页面整体的PV曲线

In [7]:
df_data = (
    df_merge
        .groupby("date")
        .agg(
            home_page=("home_page", lambda x : x.dropna().size),
            search_page=("search_page", lambda x : x.dropna().size),
            payment_page=("payment_page", lambda x : x.dropna().size),
            confirmation_page=("confirmation_page", lambda x : x.dropna().size)
        )
)
df_data.head()

,home_page,search_page,payment_page,confirmation_page
date,,,,
2015-01-01,712,436,76,6
2015-01-02,721,447,84,5
2015-01-03,760,422,64,7
2015-01-04,713,447,76,7
2015-01-05,754,462,88,11


In [8]:
# 绘制折线图
c = (
    Line()
    .add_xaxis(df_data.index.to_list())
    .add_yaxis("home_page", df_data["home_page"].to_list())
    .add_yaxis("search_page", df_data["search_page"].to_list())
    .add_yaxis("payment_page", df_data["payment_page"].to_list())
    .add_yaxis("confirmation_page", df_data["confirmation_page"].to_list())
    .set_global_opts(title_opts=opts.TitleOpts(title="整体PV折线图"))
)
c.render_notebook()

### 3. 查看分设备的PV曲线

In [9]:
df_data = (
    df_merge
        .groupby(["date", "device"])["search_page"]
        .agg(search_page=lambda x : x.dropna().size)
        .unstack()
)
df_data.head()

search_page       
device         Desktop Mobile
date                         
2015-01-01         256    180
2015-01-02         261    186
2015-01-03         222    200
2015-01-04         249    198
2015-01-05         249    213

In [10]:
c = (
    Line()
    .add_xaxis(df_data.index.to_list())
    .add_yaxis("Desktop", df_data[("search_page", "Desktop")].to_list())
    .add_yaxis("Mobile", df_data[("search_page", "Mobile")].to_list())
    .set_global_opts(title_opts=opts.TitleOpts(title="分设备PV趋势图"))
)
c.render_notebook()